**Import Libraries**

In [ ]:
import pymysql as sql
import numpy as np
import pandas as pd
import pygsheets as sheets
import getpass 

**Connect to Local F1DB**

In [ ]:
password = getpass.getpass()
f1 = sql.connect('localhost', 'root', password, 'f1db')

**Read Tables**

In [ ]:
races_tbl = pd.read_sql('SELECT * FROM races', f1)
circuits_tbl = pd.read_sql('SELECT * FROM circuits', f1)
results_tbl = pd.read_sql('SELECT * FROM results', f1)
drivers_tbl = pd.read_sql('SELECT * FROM drivers', f1)
constructors_tbl = pd.read_sql('SELECT * FROM constructors', f1)
driverstandings_tbl = pd.read_sql('SELECT * FROM driverStandings', f1)

**Join Tables**

In [ ]:
races_circuits = pd.merge(races_tbl, circuits_tbl, on='circuitId')
racefacts = races_circuits[['raceId','year','round','circuitId','name_x','date','time','url_x','name_y','location','country','lat','lng','url_y']]
racefacts = racefacts.rename(columns={'name_x':'race_name','url_x':'race_url','name_y':'circuit_name','location':'city','url_y':'circuit_url'})

**Calculate Fastest Laps by race**
Join into racefacts

In [ ]:
results = results_tbl
results['fastestLapSpeed'] = results['fastestLapSpeed'].astype('float')
fastest_lap = results.reindex(results.groupby(['raceId'])['fastestLapSpeed'].idxmax())
fastest_lap = pd.merge(fastest_lap, drivers_tbl, on='driverId', how='left')
fastest_lap['Fastest Lap Name'] = fastest_lap['forename'] + ' '  + fastest_lap['surname']
fastest_lap = fastest_lap[['raceId', 'Fastest Lap Name','fastestLapTime','fastestLapSpeed',]]
racefacts = pd.merge(racefacts, fastest_lap, on='raceId', how='left')

**Podium results by race**

In [ ]:
results = results_tbl

first_place = results[results['positionOrder'] == 1]
second_place = results[results['positionOrder'] == 2]
third_place = results[results['positionOrder'] == 3]

first_place = first_place.drop_duplicates(subset=['raceId'])
second_place = second_place.drop_duplicates(subset=['raceId'])
third_place = third_place.drop_duplicates(subset=['raceId'])

first_place = pd.merge(first_place, drivers_tbl, on='driverId', how='left')
second_place = pd.merge(second_place , drivers_tbl, on='driverId', how='left')
third_place = pd.merge(third_place, drivers_tbl, on='driverId', how='left')

first_place['First Place Name'] = first_place['forename'] + ' '  + first_place['surname']
second_place['Second Place Name'] = second_place['forename'] + ' '  + second_place['surname']
third_place['Third Place Name'] = third_place['forename'] + ' '  + third_place['surname']

first_place = first_place[['raceId', 'First Place Name','milliseconds']]
second_place = second_place[['raceId', 'Second Place Name','milliseconds']]
third_place = third_place[['raceId', 'Third Place Name','milliseconds',]]

racefacts = pd.merge(racefacts, first_place, on='raceId', how='left')
racefacts = pd.merge(racefacts, second_place, on='raceId', how='left')
racefacts = pd.merge(racefacts, third_place, on='raceId', how='left')

**Display Podium Results by Constructor**

In [ ]:
results = results_tbl
constructors = constructors_tbl

results['points'] = results['points'].astype('float')

constructor_points = results.groupby(['raceId','constructorId'])['points'].sum()
top_race_constructors = constructor_points.groupby(['raceId']).nlargest(2).to_frame(name ='points').reset_index(level=0,drop=True)

top_race_constructors = top_race_constructors.reset_index()
top_race_constructors['ranking']= top_race_constructors.groupby(['raceId']).cumcount()+1

top_race_constructors = pd.merge(top_race_constructors, constructors, on='constructorId', how='left' )
top_race_constructors = top_race_constructors[['raceId', 'points','ranking','name']]
top_race_constructors.rename(columns={'name':'constructor'})

top_race_constructors = top_race_constructors.pivot(index='raceId',columns='ranking')
top_race_constructors.columns = ['_'.join(tuple(map(str, t))) for t in top_race_constructors.columns.values]


racefacts = pd.merge(racefacts, top_race_constructors, on='raceId', how='left')
racefacts = racefacts.rename(columns={'milliseconds_x':'First Place Time','milliseconds_y':'Second Place Time',
                                      'milliseconds':'Third Place Time','location':'city','url_y':'circuit_url',
                                      'points_1':'First Constructor Points','name_1':'First Constructor',
                                      'points_2':'Second Constructor Points','name_2':'Second Constructor'
                                     })


**Constructor points by race**

In [ ]:
races = races_tbl
constructors = constructors_tbl
constructor_points = constructor_points.reset_index()

constructor_points = pd.merge(constructor_points, races, on='raceId')
constructor_points = pd.merge(constructor_points, constructors, on='constructorId')

constructor_points = constructor_points[['raceId','points','year','round','name_x','date','name_y']]
constructor_points = constructor_points.rename(columns={'name_x':'Race Name','name_y':'Constructor Name'})
constructor_points = constructor_points[constructor_points.points != 0]

**Driver points by race**

In [ ]:
results = results_tbl
races = races_tbl
constructors = constructors_tbl
drivers = drivers_tbl
drivers['Driver Name'] = drivers['forename'] + ' '  + drivers['surname']

results = results[['resultId','raceId','driverId','constructorId','grid','positionText','positionOrder',
                  'positionOrder', 'points','milliseconds']]
races = races[['raceId','year','round','name','date']]
constructors = constructors[['constructorId','name']]
drivers = drivers[['driverId','Driver Name']]

result_details = pd.merge(results, races, on='raceId', how='left')
result_details = pd.merge(result_details, constructors, on='constructorId', how='left')
result_details = pd.merge(result_details, drivers, on='driverId', how='left')

result_details = result_details.rename(columns={'name_x':'Race Name', 'name_y' : 'Constructor Name'})
result_details['join'] = 1

**Driver Standings by Race

In [ ]:
standings = driverstandings_tbl
races = races_tbl
constructors = constructors_tbl
results = results_tbl
drivers = drivers_tbl


standings = standings[['driverStandingsId','raceId','driverId','points','position','wins']]
races = races[['raceId','year','round','name','date']]
drivers['Driver Name'] = drivers['forename'] + ' '  + drivers['surname']
drivers = drivers[['driverId','Driver Name']]

standings = pd.merge(standings, races, on='raceId', how='left')
standings = pd.merge(standings, drivers, on='driverId', how='left')

results['unique'] = results['raceId'].astype('str')+ ' ' + results['driverId'].astype('str')
results = results[['resultId','unique','constructorId']]
results = results.drop_duplicates(subset=['unique'])

standings['unique'] = standings['raceId'].astype('str')+ ' ' + standings['driverId'].astype('str')
standings = pd.merge(standings, results, on='unique', how='left')

constructors = constructors[['constructorId','name']]
standings = pd.merge(standings, constructors, on='constructorId', how='left')

standings = standings.rename(columns={'name_x':'Race Name'})
standings['join'] = 1

**Publish to Google Sheets**

In [ ]:
gc = sheets.authorize(service_file='/home/x220/Documents/service-accounts/service_account_credentials.json')
sh = gc.open('f1db')
wks = sh[0]
wks.rows = racefacts.shape[0]
wks.set_dataframe(racefacts,(1,1))

wks = sh[1]
wks.rows = constructor_points.shape[0]
wks.set_dataframe(constructor_points,(1,1))

wks = sh[2]
wks.rows = result_details.shape[0]
wks.set_dataframe(result_details,(1,1))

wks = sh[3]
wks.rows = standings.shape[0]
wks.set_dataframe(standings,(1,1))

**Filter/Sort/Display**

In [ ]:
#races_circuits_results = pd.merge(races_circuits, results_tbl, on='raceId')
#racedetails = pd.merge(races_circuits_results, drivers_tbl, on='driverId')

In [ ]:
##races_circuits = races_circuits[['raceId','year','round','circuitId','name_x','date','time','url_x']]
##races_circuits.sort_values(by=['year','round'])
##display(racedetails.sort_values(by=['year','round']))